Let's query some interesting stuff about foods

In [1]:
import pandas as pd # for loading data
import mysql.connector as con


In [2]:
connection = con.connect(user = 'root', password = 'root', database = 'foodb')

Let's query some interesting data about foods

In [73]:
foods = pd.read_sql("SELECT f.id, f.name, f.food_subgroup, COUNT(DISTINCT c.source_id) AS nnutrients FROM foods AS f, contents AS c " + 
                    "WHERE f.id = c.food_id AND c.source_type = \"Nutrient\" GROUP BY f.id", connection)

In [41]:
foods.head()

,id,name,food_subgroup,nnutrients
0,2,Savoy cabbage,Cabbages,7
1,4,Kiwi,Tropical fruits,17
2,5,Allium (Onion),Onion-family vegetables,33
3,6,Garden onion,Onion-family vegetables,3
4,7,Leek,Onion-family vegetables,7


I will query the top 20 compounds that are present in most of the foods and add them to the foods table

In [11]:
compounds = pd.read_sql("SELECT ta.source_id, comp.name, ta.cnt FROM (SELECT source_id, COUNT(*) AS cnt " + 
                    "FROM `contents` WHERE source_type = \"Compound\" GROUP BY source_id ORDER BY cnt DESC LIMIT 20) ta, compounds " + 
                    "AS comp WHERE ta.source_id = comp.id" , connection)

In [12]:
compounds.head()

,source_id,name,cnt
0,14507,Folic acid,28009
1,13831,Retinol,22931
2,21595,Unsaturated fatty acids,18776
3,13267,Cyanocobalamin,12424
4,565,alpha-Tocopherol,11536


Let's rename some of the entries, so we can recognize them more easily

In [36]:
compounds.name = compounds.name.replace(
    ["Retinol", 
     "Cyanocobalamin", 
     "alpha-Tocopherol", 
     "L-Ascorbic acid", 
     "Riboflavine"], 
    ["VitaminA1", "VitaminB12", 
     "VitaminE", "VitaminC", "VitaminB2"]).str.replace(' ', '')

In [37]:
compounds.head()

,source_id,name,cnt
0,14507,Folicacid,28009
1,13831,VitaminA1,22931
2,21595,Unsaturatedfattyacids,18776
3,13267,VitaminB12,12424
4,565,VitaminE,11536


In [75]:
for _, row in compounds.iterrows():
    
    df = pd.read_sql("SELECT food_id, standard_content FROM `contents` " + 
                     "WHERE source_type = \"Compound\" AND source_id = " + str(row['source_id'])
                     , connection)
    grouped = df.groupby('food_id')
    df = grouped.median().reset_index()
    df.columns = ['food_id', row['name']]
    foods = foods.merge(df, 'left', left_on = 'id', right_on = 'food_id')
    del foods['food_id'] # remove extra column

In [76]:
foods.head()

,id,name,food_subgroup,nnutrients,Folicacid_x,VitaminA1_x,Folicacid_y,VitaminA1_y,Unsaturatedfattyacids,VitaminB12,...,Calcium,Ash,Iron,Sodium,Potassium,Phosphorus,Cholesterol,Magnesium,VitaminB2,Zinc
0,2,Savoy cabbage,Cabbages,7,0.0800,0.0,0.080,25.0,3.5,0.0,...,35.0,800.0,0.400000,28.000000,230.000000,42.000000,0.0,28.000000,0.0300,0.270000
1,4,Kiwi,Tropical fruits,17,0.0295,0.0,0.034,4.0,73.5,0.0,...,32.9,730.0,0.300000,3.000000,314.000000,33.400000,0.0,15.500000,0.0355,0.100000
2,5,Allium (Onion),Onion-family vegetables,33,0.0150,0.0,0.020,0.0,36.5,0.0,...,27.0,800.0,0.340000,16.000000,146.000000,33.000000,0.0,10.000000,0.0270,0.210000
3,6,Garden onion,Onion-family vegetables,3,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,4100.0,3.598983,3.696172,147.662253,37.427203,NaN,11.670148,0.7700,1.475484
4,7,Leek,Onion-family vegetables,7,0.0820,0.0,0.082,8.9,107.0,0.0,...,42.3,800.0,0.900000,9.000000,236.000000,45.600000,0.0,10.000000,0.1000,0.290000


In [80]:
foods.corr()

,id,nnutrients,Folicacid_x,VitaminA1_x,Folicacid_y,VitaminA1_y,Unsaturatedfattyacids,VitaminB12,VitaminE,Nicotinicacid,...,Calcium,Ash,Iron,Sodium,Potassium,Phosphorus,Cholesterol,Magnesium,VitaminB2,Zinc
id,1.000000,0.108851,-0.157018,0.116791,-0.159670,0.009095,0.057574,0.026087,-0.063446,0.017056,...,-0.170034,0.002118,-0.188178,0.108624,-0.243533,-0.056087,0.146076,-0.254864,-0.047472,-0.040327
nnutrients,0.108851,1.000000,-0.026225,0.020373,0.001519,-0.114797,0.096809,-0.068384,-0.028896,0.046959,...,-0.060871,-0.029050,-0.102360,0.011091,-0.127321,0.123639,-0.004266,-0.045639,-0.035482,0.005298
Folicacid_x,-0.157018,-0.026225,1.000000,-0.054751,0.872959,0.132756,-0.014452,-0.077147,0.038200,0.072745,...,0.471849,0.172034,0.285704,-0.003518,0.417359,0.137091,-0.137216,0.459705,0.256385,0.050664
VitaminA1_x,0.116791,0.020373,-0.054751,1.000000,-0.048318,0.592842,0.147581,0.249071,0.064104,0.092622,...,-0.017584,-0.013151,-0.028412,0.013232,-0.067764,0.078654,0.291093,-0.074862,0.074589,0.039188
Folicacid_y,-0.159670,0.001519,0.872959,-0.048318,1.000000,0.096337,-0.027479,-0.083105,0.072958,0.079691,...,0.422294,0.155963,0.312807,-0.013478,0.409463,0.182279,-0.142117,0.465626,0.248559,0.063007
VitaminA1_y,0.009095,-0.114797,0.132756,0.592842,0.096337,1.000000,0.094337,0.092967,0.041905,-0.007252,...,0.013266,-0.011137,0.015268,-0.013655,-0.005808,-0.034851,0.119164,0.022354,0.083198,-0.000653
Unsaturatedfattyacids,0.057574,0.096809,-0.014452,0.147581,-0.027479,0.094337,1.000000,-0.057398,0.459830,-0.029815,...,0.004411,-0.009925,-0.007841,-0.014488,-0.034925,0.146856,0.032229,0.172705,-0.044439,0.037679
VitaminB12,0.026087,-0.068384,-0.077147,0.249071,-0.083105,0.092967,-0.057398,1.000000,-0.010801,0.276630,...,-0.035830,-0.009746,-0.022844,-0.000478,-0.028114,0.249364,0.414130,-0.063802,0.126268,0.469094
VitaminE,-0.063446,-0.028896,0.038200,0.064104,0.072958,0.041905,0.459830,-0.010801,1.000000,0.020036,...,0.121950,0.043085,0.116004,-0.019411,0.139163,0.162844,-0.041407,0.179455,0.044218,0.055746
Nicotinicacid,0.017056,0.046959,0.072745,0.092622,0.079691,-0.007252,-0.029815,0.276630,0.020036,1.000000,...,0.129164,0.092931,0.161418,-0.024082,0.198330,0.398230,0.224559,0.145967,0.242228,0.110493
